### Import packages

In [1]:
import pandas as pd
import requests
import http.client
import mimetypes
import base64
import ast
import os
import json
import sqlite3
from bs4 import BeautifulSoup
import fasttext

### Get API access token

In [2]:
#create the Base64 encoded basic authorization string
clientID = '2bf6d937de32cac8d96791bbe0069259'
clientsecret = 'VlzWvrSjFntRuBaTA1IilLIpLRkoV9iMpqiP4wj8nQRkY1BHdA9rpWq4RB7ZiQJA'
auth = "{0}:{1}".format(clientID, clientsecret)
encodedBytes=base64.b64encode(auth.encode("utf-8"))
authStr = str(encodedBytes, "utf-8")

#create the connection 
conn = http.client.HTTPSConnection("iad1.qualtrics.com")
body = "grant_type=client_credentials"
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}
headers['Authorization'] = 'Basic {0}'.format(authStr)

#make the request
conn.request("POST", "/oauth2/token", body, headers)
res = conn.getresponse()
data = res.read()
dict_str = data.decode("UTF-8")
mydata = ast.literal_eval(dict_str)
access_token = mydata['access_token']
apiToken = 'yIzlZBENgbbLOUKR6Snc3ikZ0QiQVxKXi5YSdEgr'

### Test API connection

In [37]:
# conn = http.client.HTTPSConnection("iad1.qualtrics.com")
# body = ''
# headers = {
#   'Authorization': 'Bearer ' + access_token
# }

# # make the request
# conn.request("GET", "/API/v3/whoami", body, headers)
# res = conn.getresponse()
# data = res.read()
# print(data.decode("utf-8"))

{"result":{"brandId":"duke","userId":"UR_3Ucmdd4qzbl0gRL","userName":"sd363@duke.edu#duke","accountType":"UT_6PDrWJo2ZeDbc4k","firstName":"Steven","lastName":"Doerstling","email":"steven.doerstling@duke.edu","datacenter":"viawest"},"meta":{"requestId":"a5d9bb59-b668-43f4-8259-222ba5e24267","httpStatus":"200 - OK","notice":"Request proxied. For faster response times, use this host instead: ca1.qualtrics.com"}}


### Create new survey

In [3]:
QUALTRICS_TOKEN = apiToken

QUALTRICS_API_URL = "https://ca1.qualtrics.com/API/v3/"
API_CALL_HEADERS = {
    "x-api-token": QUALTRICS_TOKEN,
    "content-type": "application/json",
    "Accept": "application/json"
}

url = QUALTRICS_API_URL + "survey-definitions"
survey_data = {"SurveyName": "Crowdfunding - Manual Label Round 6", "Language": "EN", "ProjectCategory": "CORE"}
r = requests.post(url, json=survey_data, headers=API_CALL_HEADERS)
r.raise_for_status()
res = r.json()["result"]
print(f"Response survey create: {res}")
survey_id = res["SurveyID"]

Response survey create: {'SurveyID': 'SV_0BVi2Lg8EGbxqO9', 'DefaultBlockID': 'BL_eyS4vre6PoNT5MF'}


### Import campaign data to populate questions

In [7]:
conn = sqlite3.connect("gfm.db")
feed = pd.read_sql_query("SELECT * FROM feed_tb", conn)

In [8]:
feed = feed.drop_duplicates('url')
feed['fund_description'] = [x.replace('\n','') for x in feed['fund_description']]
feed['nchar']= [len(x) for x in feed['fund_description']]
feed = feed[feed['nchar'] >= 100]

In [9]:
def GetLanguage(df):
    path_to_pretrained_model = 'fasttext_models/lid.176.bin'
    fmodel = fasttext.load_model(path_to_pretrained_model)
    r = fmodel.predict(feed['fund_description'].to_list())
    labels = r[0]
    labels = [i[0].split('__')[2] for i in labels]
    scores = r[1]
    scores = [i[0] for i in scores]
    return labels, scores

In [10]:
feed['language'], feed['lang_score'] = GetLanguage(feed)

In [11]:
feed = feed[feed['language'] == 'en']

In [12]:
len(feed)

89676

Make sure that there are no duplicate urls from previous reviews

In [10]:
round_1 = pd.read_csv("Manual_review/round 1/example_50.csv")
round_1 = round_1[['url','fund_description']]

In [11]:
round_2 = pd.read_csv("Manual_review/round 2/example_51-100.csv")
round_2 = round_2[['url','fund_description']]

In [12]:
round_3 = pd.read_csv("Manual_Review/round 3/example_101-150.csv")
round_3 = round_3[['url','fund_description']]

In [13]:
round_4 = pd.read_csv("Manual_Review/round 4/example_151-200.csv")
round_4 = round_4[['url','fund_description']]

In [15]:
round_5 = pd.read_csv("Manual_Review/round 5/example_201-250.csv")
round_5 = round_5[['url','fund_description']]

In [16]:
feed = feed[~feed['url'].isin(round_1['url'])]
feed = feed[~feed['url'].isin(round_2['url'])]
feed = feed[~feed['url'].isin(round_3['url'])]
feed = feed[~feed['url'].isin(round_4['url'])]
feed = feed[~feed['url'].isin(round_5['url'])]

In [22]:
feed = feed[['url','fund_description']].sample(50)

In [23]:
feed.to_csv("Manual_review/round 5/example_251-300.csv", index=False)

### Generate survey questions

#### Upload these using online interface and delete default question block

In [24]:
results = []
for index, row in feed.iterrows():
    doc = '''
    [[AdvancedFormat]]

    [[Block:{0}]]
    
    [[Question:MC:MultipleAnswer:Vertical]]
    [[ID:{0}__base]]
    {1}
    
    <br />
    <br />
    ------------------------------------------------
    <br />
    ------------------------------------------------
    <br />
    Please select all disease categories represented.
    [[Choices]]
    Cardiovascular diseases
    Endocrine diseases
    Gastrointestinal diseases
    Genitourinary disorders
    Infections
    Injuries and external causes
    Mental and substance use disorders
    Musculoskeletal diseases
    Neoplasms
    Nervous system diseases
    Nonspecific
    Other noncommunicable diseases
    Pregnancy and childbirth
    Respiratory diseases
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:{0}__top]]
    Please select the primary disease category.
    [[Choices]]
    Cardiovascular diseases
    Endocrine diseases
    Gastrointestinal diseases
    Genitourinary disorders
    Infections
    Injuries and external causes
    Mental and substance use disorders
    Musculoskeletal diseases
    Neoplasms
    Nervous system diseases
    Nonspecific
    Other noncommunicable diseases
    Pregnancy and childbirth
    Respiratory diseases
    
    [[Question:TE]]
    [[ID:{0}__NOTES]]
    Please make any notes below.

    [[PageBreak]]

    '''.format(row['url'],row['fund_description'])
    results.append(doc)

doc = "\n".join(results)

with open("Manual_review/round 6/survey_251-300.txt", "w") as text_file:
    text_file.write(doc)

# Export data for super earners

In [4]:
import pandas as pd
import requests
import http.client
import mimetypes
import base64
import ast
import os
import json
import sqlite3
from bs4 import BeautifulSoup
import fasttext

In [5]:
feed = pd.read_csv('Super earners/data/sample_20.csv')

In [6]:
feed = feed.drop_duplicates('url')
feed['fund_description'] = [x.replace('\n','') for x in feed['fund_description']]
feed['nchar']= [len(x) for x in feed['fund_description']]
feed = feed[feed['nchar'] >= 100]

In [7]:
def GetLanguage(df):
    path_to_pretrained_model = 'fasttext_models/lid.176.bin'
    fmodel = fasttext.load_model(path_to_pretrained_model)
    r = fmodel.predict(feed['fund_description'].to_list())
    labels = r[0]
    labels = [i[0].split('__')[2] for i in labels]
    scores = r[1]
    scores = [i[0] for i in scores]
    return labels, scores

In [8]:
feed['language'], feed['lang_score'] = GetLanguage(feed)

In [9]:
feed = feed[feed['language'] == 'en']

In [10]:
len(feed)

20

In [11]:
results = []
for index, row in feed.iterrows():
    doc = '''
    [[AdvancedFormat]]

    [[Block:{0}]]
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:individual_or_organization]]
    
    <h1>{1}</h1>
    
    <br />
    <br />
    
    <img src={2} alt="Flowers in Chania">
    
    <br />
    <br />
    
    {3}
    
    <br />
    <br />
    ------------------------------------------------
    <br />
    ------------------------------------------------
    <br />
     ------------------------------------------------
    <br />
    ------------------------------------------------
    <br />
    
    
    Is this campaign on behalf of an individual or organization
    [[Choices]]
    Individual
    Organization
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:links_external]]
    Are there links to external websites or news articles?
    [[Choices]]
    Yes
    No
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:appeals]]
    Is there at least one appeal to share the campaign (by social media, email, word of mouth, etc.)?
    [[Choices]]
    Yes
    No
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:author_relationship]]
    What is the author's relationship to the campaign beneficiary?
    [[Choices]]
    Written by the beneficiary (first-person)
    Parent
    Spouse/partner
    Child
    Sibling
    Friend(s)
    Stranged
    Extended family
    Co-worker or community member
    Other relationship (not otherwise specificied)
    Unknown
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:patient_culpability]]
    Is there mention of the patient's culpability for the illness?
    [[Choices]]
    "It's not their fault"
    Takes responsibility
    No mention of culpability
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:age]]
    What is the patient's age?
    [[Choices]]
    Pediatric
    Adult
    Unclear
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:gender]]
    What is the patient's gender?
    [[Choices]]
    Female
    Male
    Other
    
    [[Question:MC:MultipleAnswer:Vertical]]
    [[ID:family_role]]
    What is the patient's family role? Select all that apply.
    [[Choices]]
    Parent
    Grandparent
    Spouse
    Child
    Sibling
    Other
    No mention of family role
    
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:career]]
    Does the campaign mention the patient's job or career?
    [[Choices]]
    Yes
    No
    
    [[Question:TE]]
    [[ID:career_notes]]
    If there is mention of job/career, please specify below.
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:positive_char_ref_patient]]
    Is there a positive character reference for the patient?
    [[Choices]]
    No
    Yes - one
    Yes - more than one
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:positive_char_ref_other]]
    Is there a positive character reference for the caregiver or family member of the patient?
    [[Choices]]
    No
    Yes - one
    Yes - more than one
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:insurance]]
    What is the patient's insurance status?
    [[Choices]]
    Currently insured
    Currently uninsured
    Unclear
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:mention_funds]]
    Does the campaign describe the intended use of funds or a specific financial burden?
    [[Choices]]
    Yes
    No

    [[Question:MC:MultipleAnswer:Vertical]]
    [[ID:non_medical_funds]]
    Does the campaign describe intended use of funds or a financial burden related to <strong>non-medical</strong> expenses? Select all that apply.
    [[Choices]]
    Accounting for lost earnings
    Travel
    Child care
    Other non-medical expense (not otherwise specified)
    Does not mention non-medical expenses or financial burden
    
    [[Question:MC:MultipleAnswer:Vertical]]
    [[ID:medical_funds]]
    Does the campaign mention intended use of funds or a financial burden related to <strong>medical</strong> expenses? Select all that apply.
    [[Choices]]
    Medical expenses - no mention of insurance coverage
    Medical expenses - partly covered by insurance
    Medial expenses - not covered by insurance
    Medical expenses (not otherwise specified)
    Does not mention medical expeneses or financial burden
    
    [[Question:MC:SingleAnswer:Vertical]]
    [[ID:other_funds]]
    Does the campaign mention intended use of funds for <strong>unclear/unspecified</strong> purposes?
    [[Choices]]
    Yes
    No
    
    [[Question:TE]]
    [[ID:notes]]
    Feel free to make any notes below.


    [[PageBreak]]

    '''.format(
        row['url'],
        row['fund_name'],
        row['campaign_photo_url'],
        row['fund_description'])
    results.append(doc)


In [12]:
doc = "\n".join(results)

with open("Super earners/super_earners_sample_20_latest.txt", "w") as text_file:
    text_file.write(doc)

### Now import survey .txt file to qualtrics.com and delete default question block

### -------------------------------

### EVERYTHING BELOW IS NOW OLD CODE

# Export data for manual NER Review in prodigy

In [25]:
from bs4 import BeautifulSoup as bs
import re
import sqlite3
import pandas as pd
import fasttext

In [59]:
conn = sqlite3.connect("gfm.db")
feed = pd.read_sql_query("SELECT * FROM feed_tb", conn)

In [60]:
feed = feed.drop_duplicates('url')

feed['nchar']= [len(x) for x in feed['fund_description']]
feed = feed[feed['nchar'] >= 100]

#regular expression to remove url
url_reg = r'(?:(?:http|https):\/\/)?([-a-zA-Z0-9.]{2,256}\.[a-z]{2,4})\b(?:\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?'

#regular expression to remove emojis
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

feed['fund_description'] = [bs(x).get_text().replace("\n","").replace(u'\xa0', u' ') for x in feed['fund_description']]
feed['fund_description'] = [re.sub(url_reg,"",x) for x in feed['fund_description']]
feed['fund_description'] = [emoji_pattern.sub(r'', x) for x in feed['fund_description']]
feed['fund_description'] = [x.strip() for x in feed['fund_description']]


feed['nchar']= [len(x) for x in feed['fund_description']]
feed = feed[feed['nchar'] >= 100]

In [61]:
def GetLanguage(df):
    path_to_pretrained_model = 'fasttext_models/lid.176.bin'
    fmodel = fasttext.load_model(path_to_pretrained_model)
    r = fmodel.predict(feed['fund_description'].to_list())
    labels = r[0]
    labels = [i[0].split('__')[2] for i in labels]
    scores = r[1]
    scores = [i[0] for i in scores]
    return labels, scores

In [62]:
feed['language'], feed['lang_score'] = GetLanguage(feed)

In [63]:
feed = feed[feed['language'] == 'en']

In [64]:
len(feed)

89493

In [65]:
round_3 = pd.read_csv("Manual_Review/round 3/example_101-150.csv")
round_4 = pd.read_csv("Manual_Review/round 4/example_151-200.csv")
round_5 = pd.read_csv("Manual_Review/round 5/example_201-250.csv")
round_6 = pd.read_csv("Manual_Review/round 6/example_251-300.csv")

In [66]:
urls = round_3['url'].tolist() + round_4['url'].tolist() + round_5['url'].tolist() + round_6['url'].tolist()

In [67]:
df = feed[feed['url'].isin(urls)][['url','fund_description']]

In [68]:
for url in urls:
    if df['url'].str.contains(url).any():
        continue
    else:
        print(url)

https://www.gofundme.com/f/evp227-fernando-diaz
https://www.gofundme.com/f/esperanza-para-mi-viejito-transplante-de-pulmon
https://www.gofundme.com/f/2ujjqk


In [69]:
import json
df = df.rename(columns={'fund_description':'text'})
df.to_json('json/rounds_3-6_for_prodigy.json', orient="records")

### Get survey data

In [41]:
conn = http.client.HTTPSConnection("ca1.qualtrics.com")

headers = { 'x-api-token': apiToken}

conn.request("GET", "/API/v3/survey-definitions/{0}".format(survey_id), headers=headers)

res = conn.getresponse()
data = res.read()


In [42]:
a = json.loads(data.decode("utf-8"))['result']['Questions']
qids = [int(x[3:]) for x in a.keys()]
#question id1 is from the deleted default block
main_qids = range(2, int(max(qids)),17)

### Add display logic to questions

In [43]:
for main_qid in main_qids:
    
    headers = {
       'accept': "application/json",
       'content-type': "application/json",
       "x-api-token": apiToken,
    }
    

    #loop through nested categories
    for n in range(1,16):
        qid = main_qid + n + 1 #added +1 now that notes option is added after main question
        questionID = "QID{0}".format(qid)
        mainId = "QID{0}".format(main_qid)

        baseUrl = "https://{0}.qualtrics.com/API/v3/survey-definitions/{1}/questions/{2}".format(
        'ca1', survey_id, questionID)

        data = a[questionID]
        data['DisplayLogic'] = {
            "0": {
              "0": {
                "LogicType": "Question",
                "QuestionID": "QID{0}".format(main_qid),
                "QuestionIsInLoop": "no",
                "ChoiceLocator": "q://QID{0}/SelectableChoice/{1}".format(main_qid, n),
                "Operator": "Selected",
                "QuestionIDFromLocator": "QID{0}".format(main_qid),
                "LeftOperand": "q://QID{0}/SelectableChoice/{1}".format(main_qid, n),
                "Type": "Expression",
                "Description": '''<span class="ConjDesc">If</span> <span class="QuestionDesc">{0}</span> <span class="LeftOpDesc">CHOICE</span> <span class="OpDesc">Is Selected</span>'''.format(a[mainId]["QuestionText"][:97])
              },
              "Type": "If"
            },
            "Type": "BooleanExpression",
            "inPage": True
        }


        response = requests.put(baseUrl, json=data, headers=headers)